In [55]:
using JuMP 
using Gurobi 
using CSV 
using LinearAlgebra
using DataFrames
using Random
using Statistics, ScikitLearn
@sk_import metrics: roc_auc_score

PyObject <function roc_auc_score at 0x7fd998b1e5e0>

In [27]:
gurobi_env = Gurobi.Env()

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x00007fd98e4d2600)

### Stable Robust Logistic Regression

In [28]:
function classification_metrics(preds, actual)
    accuracy = sum(preds .== actual)/size(preds)[1]
    tpr = dot(
        (preds.==1),actual.==1
        )/(
        dot((preds.==1),actual.==1
            ) + dot(
            (preds.==-1),actual.==1)
    )
    fpr = dot(
        (preds.==1),actual.==-1
        )/ (
        dot((preds.==1),actual.==-1
            ) + dot(
            (preds.==-1),actual.==-1)
    )
    return accuracy, tpr, fpr
end


classification_metrics (generic function with 1 method)

In [29]:
function one_hot_encode(X, names)
    X2 = deepcopy(X)
    select!(X2, Not(Symbol.(names)))
    for i in names
        vales = unique(X[i])
        for j in 1:length(vales)-1
           X2[Symbol(string(i)*"_"*string(vales[j]))] = (X[i].==vales[j])*1
        end
    end
    return X2
end

function normalize(X, names)
    X2 = deepcopy(X)
    select!(X2, Not(Symbol.(names)))
    for j in names
        X2[j] = (X[:,j] .- mean(X[:,j])) / std(X[:,j])
    end
    return X2
end

function clean(X)
    n,p = size(X)
    i = 0
    while i < n
        i += 1
        if "?" in X[i,:]
            X = X[1:end .!= i, :]
            i -= 1
        end
        n,_ = size(X)
    end
    return X
end     

function toNum(df, names)
    n,p = size(df)
    for name in names
        if !(isa(df[1,name], Int64) || isa(df[1,name], Float64))
            temp = zeros(n)
            for i=1:n
                temp[i] = parse(Float64,df[i,name])
            end
            df[!,name] = temp
        end
    end
    return df
end

function preprocess(df, categorical_vars, numerical_vars)
    df = clean(df)
    df = toNum(df,numerical_vars)
    df = normalize(df,numerical_vars)
    df = one_hot_encode(df[:,1:end], categorical_vars) 
    df[df[:,end].==0,end] .= -1
    return df
end

preprocess (generic function with 1 method)

In [118]:
function compute_∇f(w_k, y, X, λ)
    n, p = size(X)
    temp = zeros(p)
    for i in 1:n
        t = exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k)
#       t = exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k)

        if t == Inf
            Δ = -y[i]*Array(X[i,:]) .+ 2*λ*w_k
        else
            Δ = (1/(1+t))*t*(-y[i]*Array(X[i,:]) .+ 2*λ*w_k)
        end
        temp = temp + Δ
    end
    ∇f_k = temp
    return ∇f_k
end

function compute_f(w_k, y, X, λ)
    n, p = size(X)
    taylor = 0
    f_k = 0
    for i in 1:n
        t = exp(-y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k)
        if t == Inf 
            f_k += -y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k
        else
            f_k += log(1+t)
        end
    end

    return f_k
end

compute_f (generic function with 1 method)

In [119]:

function compute_∇f_s(w_k, z, y, X, λ)
    n, p = size(X)
    temp = zeros(p)
    for i in 1:n
        t = exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k)

        if t == Inf
            Δ = z[i]*(-y[i]*Array(X[i,:]) .+ 2*λ*w_k)
        else
            Δ = (z[i]/(1+t))*t*(-y[i]*Array(X[i,:]) .+ 2*λ*w_k)
        end
        temp = temp + Δ
    end
    ∇f_k = temp
    return ∇f_k
end    

function inner(w_k, y, X, k, λ)
    
    n, p = size(X)
    
    model_inner = Model(solver=GurobiSolver(OutputFlag=0,gurobi_env))
    
    @variable(model_inner, 1 >= z[1:n] >= 0)
    
    @constraint(model_inner, sum(z) <= k)
    
    exponents = []
    
    for i in 1:n
        expo = log(1+exp(-y[i]*dot(X[i,:], w_k)+ λ*dot(w_k,w_k)))
        if expo == Inf
            expo = -y[i]*dot(X[i,:], w_k)+ λ*dot(w_k,w_k)
        end
        append!(exponents, expo)
    end
    
    @objective(model_inner,
        Max,
        sum(z[i]*exponents[i] for i=1:n)
    )
    
    
    solve(model_inner)
    
    optimal_z = getvalue(z)
    optimal_f = getobjectivevalue(model_inner)
    
    return optimal_z, optimal_f
    
end

inner (generic function with 1 method)

In [120]:
### Cutting Planes Implementation ###
function rlr(y, X, ε, λ)
    errors = []
    n, p = size(X)
    # Initialization values and step 0
    w_0 = [0 for i in 1:p]
    #w_0 = [rand(Uniform(-0.5, 0.5)) for i in 1:p]
    f_0 = sum(log(1+exp(-y[i]*dot(X[i,:], w_0)+λ*transpose(w_0)*w_0)) for i=1:n)
    ∇f_0 = compute_∇f(w_0, y, X, λ)

    # Outer minimization problem
    outer_min_model = Model(solver=GurobiSolver(OutputFlag=0, gurobi_env))
    @variable(outer_min_model, t >= 0)
    @variable(outer_min_model, w[1:p])
    #@constraint(outer_min_model, [j=1:p], -1 <= w[j] <= 1)
    @constraint(outer_min_model, t >= f_0 + (dot(∇f_0, w)-dot(∇f_0, w_0)))
    @constraint(outer_min_model, [j=1:p], 10 >= w[j])
    @constraint(outer_min_model, [j=1:p], w[j] >= -10)
    @objective(outer_min_model, Min, t)
    k = 1 # Number of constraints in the final problem
    solve(outer_min_model)

    # New steps k
    t_k = getvalue(t)
    w_k = getvalue(w)

    f_k = compute_f(w_k, y, X, λ)
    #f_k = sum(min(log(1+exp(-y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k)),taylor) for i=1:n)
    
    ∇f_k = compute_∇f(w_k, y, X, λ)
    
    while abs(f_k - t_k) >= ε # error

        push!(errors, f_k - t_k)
        @constraint(outer_min_model,t >= f_k +(dot(∇f_k, w)-dot(∇f_k, w_k)))
        k += 1
        solve(outer_min_model)
        # Updating all the values
        t_k = getvalue(t)
        w_k = getvalue(w)
        
        f_k = compute_f(w_k, y, X, λ)
        #f_k = sum(min(log(1+exp(-y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k)),100) for i=1:n)

        ∇f_k = compute_∇f(w_k, y, X, λ)
         if k%500 == 0
             println("Number of constraints: ", k, "\t Error = ", abs(t_k - f_k))
         end
        if k > 20000
            break
        end
    end
    push!(errors, f_k - t_k)
    return t_k, f_k, w_k, errors
end

function srlr(y, X, epsilon, k, λ)
    deltas = []
    n, p = size(X)
    initialization_w = [0 for i in 1:p]
    initialization_z, initial_f = inner(initialization_w, y, X, k, λ)
    initial_derivative_f = compute_∇f_s(initialization_w, initialization_z, y, X, λ)
    
    model_outer = Model(solver=GurobiSolver(OutputFlag=0, gurobi_env))
    
    @variable(model_outer, t >= 0)
    @variable(model_outer, w[1:p])
    
    @constraint(
        model_outer, t >= initial_f + dot(initial_derivative_f, w)-dot(initial_derivative_f, initialization_w)
    )
    @constraint(model_outer, [j=1:p], 10 >= w[j])
    @constraint(model_outer, [j=1:p], w[j] >= -10)
    
    @objective(model_outer, Min, t)
    
    number_const = 1
    solve(model_outer)

    t_new = getvalue(t)
    w_new = getvalue(w)
    z_new, f_new = inner(w_new, y, X, k, λ)

    derivative_f_new = compute_∇f_s(w_new, z_new, y, X, λ)
    while abs(f_new - t_new) >= epsilon
        
        push!(deltas, f_new - t_new)
        
        @constraint(model_outer,t >= f_new + dot(derivative_f_new, w)-dot(derivative_f_new, w_new))
        
        number_const += 1
        solve(model_outer)
        t_new = getvalue(t)
        w_new = getvalue(w)
        z_new, f_new = inner(w_new, y, X, k, λ)

        derivative_f_new = compute_∇f_s(w_new, z_new, y, X, λ)
        
        if number_const%100 == 0
            println("Number of constraints: ", number_const, "\t Step delta = ", abs(t_new - f_new))
        end
        
        if number_const > 100000
            break
            
        end
    end
    push!(deltas, f_new - t_new)
    return t_new, f_new, w_new, z_new, deltas
end


srlr (generic function with 1 method)

In [165]:
function robust_LG_valid(X, y, ε, lambda_vals; method=srlr, split_at=0.8)
    n,p = size(X)
    k = convert(Int,floor(split_at*n))
#     permuted_indices = randperm(n)
#     train_indices, valid_indices = permuted_indices[1:split], permuted_indices[split+1:end]
#     X_train, y_train = X[train_indices,:], y[train_indices]
#     X_valid, y_valid = X[valid_indices,:], y[valid_indices]
    
#    accuracies = zeros(length(lambda_vals))
    AUCs = zeros(length(lambda_vals))
    for (i,λ) in enumerate(lambda_vals)
        println(i)
        t, f, w, z, e = srlr(y, X, ε, k, λ)
        
        train_indices = z.>0
        valid_indices = z.==0
         y_train, X_train = y[train_indices], X[train_indices,:]
         y_valid, X_valid = y[valid_indices], X[valid_indices,:]
        
#         pred = 1 ./ (1 .+ exp.(-(Matrix(X_valid)*w).+λ*transpose(w)*w)) .> 0.5
#         accuracies[i] = 1-sum(pred .!= (y_valid .== 1))/length(y_valid)
        
        pred_prob = 1 ./ (1 .+ exp.(-(Matrix(X_valid)*w).+λ*transpose(w)*w))
        AUCs[i] = roc_auc_score(y_valid ,pred_prob)
    end
    IJulia.clear_output()

    i_best = argmax(AUCs)
    t, f, w_best, e = rlr(y, X, ε, lambda_vals[i_best])
#     train_indices = z_best.>0
#     valid_indices = z_best.==0
#     y_train, X_train = y[train_indices], X[train_indices,:]
#     y_valid, X_valid = y[valid_indices], X[valid_indices,:]
    return  w_best, lambda_vals[i_best], AUCs #, y_train, y_valid , z_best,
end

robust_LG_valid (generic function with 1 method)

In [170]:
function test(df, categorical_vars, numerical_vars, test_split, validation_split, ε, lambda_vals, seed)
    Random.seed!(seed)
    start = time()
    n,_ = size(df)
    permuted_indices = randperm(n)
    test_split = convert(Int,floor(validation_split*n))
    train_indices, test_indices = permuted_indices[1:test_split], permuted_indices[test_split+1:end]
    train = df[train_indices,:]
    test = df[test_indices,:]
    train_X = train[:,1:end-1]
    train_y = train[:,end]
    test_X = test[:,1:end-1]
    test_y = test[:,end]
    
    IJulia.clear_output()
    println("Enter cross-validation")
    w, λ, AUCs  = robust_LG_valid(train_X, train_y, ε, lambda_vals; method=LR_cutting_planes, split_at=validation_split)
    elapsed = time() - start
    pred_prob = 1 ./ (1 .+ exp.(-(Matrix(test_X)*w).+λ*transpose(w)*w))
    pred = pred_prob.> 0.5
    accuracy = 1-sum(pred .!= (test_y .== 1))/length(test_y)
    auc = roc_auc_score(test_y ,pred_prob)
    IJulia.clear_output()

    return auc, accuracy, λ, elapsed, w, z, y_train, y_valid
end

test (generic function with 1 method)

In [171]:
df = CSV.read("Data/caesarian.csv";header=true)
size(df)

(80, 6)

In [172]:
#lambda_vals = [0.00001,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1]
lambda_vals = [0.00005x for x in 1:20]
categorical_vars = Symbol.(["Delivery number" ;"Delivery time";"Blood of Pressure";"Heart Problem"])
numerical_vars = Symbol.(["Age"])
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(80, 10)

In [173]:
seed = 1
auc, acc, lambda, elapsed, w, z, y_train, y_valid = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, 1)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.7575757575757576 Accuracy: 0.6 λ: 5.0e-5 Time: 4.870409965515137

In [186]:
shuffle(repeat(1:3,2))[1:end-2]

4-element Array{Int64,1}:
 1
 2
 3
 1

In [100]:
y_valid

15-element Array{Int64,1}:
  1
  1
  1
 -1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1

In [99]:
y_train

45-element Array{Int64,1}:
 -1
  1
 -1
 -1
  1
  1
  1
  1
 -1
  1
 -1
 -1
 -1
  ⋮
  1
  1
 -1
  1
 -1
 -1
 -1
  1
  1
 -1
  1
  1

In [174]:
df = CSV.read("Data/framingham.csv";header=true)
size(df)

(3658, 16)

In [175]:
categorical_vars = Symbol.(["education"])
numerical_vars = Symbol.(["age" ;"cigsPerDay";"totChol";"sysBP";"diaBP";"BMI";"heartRate";"glucose"])
lambda_vals = [0.00001,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1]
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(3658, 18)

In [176]:
seed = 1
auc, acc, lambda, elapsed, w = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, 1)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

Enter cross-validation
1
Number of constraints: 100	 Step delta = 370.4084839056186
Number of constraints: 200	 Step delta = 22.04671343915186
Number of constraints: 300	 Step delta = 1.5158617714167804
Number of constraints: 400	 Step delta = 0.19645699197951672
Number of constraints: 500	 Step delta = 0.008271670782960427
Number of constraints: 600	 Step delta = 0.0008094738016097835


PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /Users/ivanxu/.julia/packages/PyCall/BcTLp/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('Only one class present in y_true. ROC AUC score is not defined in that case.')
  File "/Users/ivanxu/.julia/conda/3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 72, in inner_f
    return f(**kwargs)
  File "/Users/ivanxu/.julia/conda/3/lib/python3.8/site-packages/sklearn/metrics/_ranking.py", line 390, in roc_auc_score
    return _average_binary_score(partial(_binary_roc_auc_score,
  File "/Users/ivanxu/.julia/conda/3/lib/python3.8/site-packages/sklearn/metrics/_base.py", line 77, in _average_binary_score
    return binary_metric(y_true, y_score, sample_weight=sample_weight)
  File "/Users/ivanxu/.julia/conda/3/lib/python3.8/site-packages/sklearn/metrics/_ranking.py", line 223, in _binary_roc_auc_score
    raise ValueError("Only one class present

In [149]:
sum(y_train .== 1)/length(y_train)

0.5605250364608654

In [105]:
y_valid

686-element Array{Int64,1}:
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
  ⋮
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1

In [106]:
sum(y_valid)

-686